In [1]:
import genanki

# === Parameters ===
deck_id = 2059400110  # large random int
model_id = 1607392319
deck_name = "Generated Flashcards"
output_filename = "generated_flashcards.apkg"
num_cards = 50

# === Define the model (Basic front/back) ===
my_model = genanki.Model(
    model_id,
    'Basic Model',
    fields=[
        {'name': 'Front'},
        {'name': 'Back'},
    ],
    templates=[{
        'name': 'Card 1',
        'qfmt': '{{Front}}',
        'afmt': '{{FrontSide}}<hr id="answer">{{Back}}',
    }]
)

# === Create the deck ===
my_deck = genanki.Deck(
    deck_id,
    deck_name
)

# === Generate flashcards ===
for i in range(num_cards):
    front = f"FRONTFILLER_{i}"
    back = f"BACKFILLER_{i}"
    note = genanki.Note(
        model=my_model,
        fields=[front, back]
    )
    my_deck.add_note(note)

# === Export to .apkg ===
genanki.Package(my_deck).write_to_file(output_filename)

print(f"✅ Exported {num_cards} flashcards to {output_filename}")


✅ Exported 50 flashcards to generated_flashcards.apkg


In [ ]:
import requests
import json
import time
import subprocess

# === Configuration ===
MODEL = "mistral"
OLLAMA_URL = "http://localhost:11434"

def print_gpu_info():
    """Print current GPU status using nvidia-smi"""
    try:
        output = subprocess.check_output([
            'nvidia-smi', '--query-gpu=name,memory.total,memory.used,utilization.gpu',
            '--format=csv,nounits,noheader'
        ])
        print("🖥️ GPU Status:")
        print("Name\t\tTotal Mem\tUsed Mem\tUtilization")
        print(output.decode().strip())
    except Exception as e:
        print("⚠️ Could not access GPU via nvidia-smi:", e)

def get_model_info(model=MODEL):
    """Query Ollama to confirm model and backend"""
    url = f"{OLLAMA_URL}/api/show"
    try:
        response = requests.post(url, json={"name": model})
        if response.ok:
            data = response.json()
            backend = data.get("details", {}).get("backend", "Unknown")
            print(f"📊 Model: {data.get('name')}")
            print(f"⚙️ Backend: {backend}")
            if "cuda" in backend.lower():
                print("🔥 GPU is active (CUDA confirmed)")
            else:
                print("❄️ CPU or fallback backend in use")
        else:
            print("❌ Model query failed:", response.status_code, response.text)
    except Exception as e:
        print("⚠️ Error querying model:", e)

def call_ollama(prompt, model=MODEL):
    """Send prompt to Ollama and log status"""
    url = f"{OLLAMA_URL}/api/generate"
    payload = {
        'model': model,
        'prompt': prompt,
        'stream': False
    }

    print(f"\n🧠 Sending prompt to '{model}'...")
    print("📤 Prompt:", prompt[:100] + "..." if len(prompt) > 100 else prompt)

    start = time.time()
    response = requests.post(url, json=payload)
    duration = time.time() - start

    print(f"📡 HTTP Status: {response.status_code}")
    if response.ok:
        try:
            data = response.json()
            if 'error' in data:
                print(f"❌ Ollama Error: {data['error']}")
            else:
                output = data['response']
                print(f"✅ Response in {duration:.2f} sec:")
                print(output.strip())
                return output.strip()
        except Exception as e:
            print("⚠️ Could not parse JSON:", e)
            print("🔍 Raw response:")
            print(response.text)
    else:
        print(f"❌ Request failed: {response.status_code}")
        print("🔍 Raw error:")
        print(response.text)

# === Run Diagnostic + Inference ===
print_gpu_info()
get_model_info()
call_ollama("Translate this sentence into Japanese with furigana: 'I want to learn how to cook.'")


🖥️ GPU Status:
Name		Total Mem	Used Mem	Utilization
NVIDIA GeForce RTX 4060 Laptop GPU, 8188, 0, 0
📊 Model: None
⚙️ Backend: Unknown
❄️ CPU or fallback backend in use

🧠 Sending prompt to 'mistral'...
📤 Prompt: Translate this sentence into Japanese with furigana: 'I want to learn how to cook.'
